<a href="https://colab.research.google.com/github/kyangwi/Machine-Learning/blob/main/Face_Recognition_using_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install dlib-bin
!pip install numpy scikit-learn opencv-python

In [3]:
import os
import cv2
import dlib
import numpy as np
from sklearn.svm import SVC
import pickle

In [4]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
# bzip2 -d shape_predictor_68_face_landmarks.dat.bz2


--2025-04-28 13:22:29--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-04-28 13:22:29--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.1’

shape_predictor_68_ 100%[===================>]  61.07M  34.9MB/s    in 1.7s    

2025-04-28 13:22:31 (34.9 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.1’ saved [64040097/64040097]



In [5]:
!wget https://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2


--2025-04-28 13:22:31--  https://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21428389 (20M)
Saving to: ‘dlib_face_recognition_resnet_model_v1.dat.bz2.1’

dlib_face_recogniti 100%[===================>]  20.44M  25.6MB/s    in 0.8s    

2025-04-28 13:22:32 (25.6 MB/s) - ‘dlib_face_recognition_resnet_model_v1.dat.bz2.1’ saved [21428389/21428389]



In [6]:
path = "/content/drive/MyDrive/facedata"

In [7]:
import bz2
comp = open("/content/shape_predictor_68_face_landmarks.dat.bz2", "rb").read()
with open("/content/shape_predictor_68_face_landmarks.dat", "wb") as f:
    f.write(bz2.decompress(comp))

In [8]:
import bz2
comp = open("/content/dlib_face_recognition_resnet_model_v1.dat.bz2", "rb").read()
with open("/content/dlib_face_recognition_resnet_model_v1.dat", "wb") as f:
    f.write(bz2.decompress(comp))

In [9]:
image_path = "/content/drive/MyDrive/facedata"

In [10]:
# 1. Load models
hog_detector = dlib.get_frontal_face_detector()
shape_pred = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")
face_rec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")


In [14]:
import tqdm
# 2. Prepare data
dataset_dir = image_path  # subfolders per person
embeddings, labels = [], []

for person in tqdm.tqdm(os.listdir(dataset_dir)):
    person_dir = os.path.join(dataset_dir, person)
    if not os.path.isdir(person_dir):
        continue
    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        bgr = cv2.imread(img_path)
        if bgr is None:
            continue
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        # Detect faces
        rects = hog_detector(rgb, 1)  # upsample once
        if not rects:
            continue
        # Assume first detected face
        aligned = dlib.get_face_chip(rgb, shape_pred(rgb, rects[0]), size=150)
        # Compute embedding
        vec = np.array(face_rec.compute_face_descriptor(aligned, num_jitters=1))
        embeddings.append(vec)
        labels.append(person)


100%|██████████| 9/9 [00:57<00:00,  6.38s/it]


In [18]:
embeddings = np.array(embeddings)
labels = np.array(labels)

In [22]:
import pandas as pd
data = pd.DataFrame(embeddings)

In [30]:
combined = pd.concat([data,pd.Series(labels, name='label')],axis=1)

In [31]:
from sklearn.utils import shuffle
combined = shuffle(combined).reset_index(drop=True)

In [32]:
combined.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,-0.121363,0.069113,0.072550,-0.111565,-0.065676,-0.050649,0.006180,-0.085427,0.159642,-0.060541,...,0.081051,-0.098120,-0.092870,-0.048838,-0.019959,0.085465,-0.082869,0.013498,0.065732,Elyse
1,-0.183620,0.070572,0.067416,-0.014060,-0.010353,-0.037324,0.073805,-0.056252,0.075191,-0.044896,...,0.134942,-0.092776,0.056739,-0.096264,-0.091645,0.036652,-0.034234,0.032303,0.083231,josh
2,-0.088148,0.082711,0.062080,-0.037792,-0.031343,-0.026676,0.020098,-0.129539,0.159337,-0.120829,...,0.067982,-0.061606,-0.044457,-0.066005,-0.050304,0.110513,-0.072543,0.042720,0.037290,Elyse
3,-0.131311,0.116146,0.046821,-0.025888,0.015320,-0.039035,0.068192,-0.106677,0.131349,-0.034599,...,0.096359,-0.100113,0.060262,-0.052179,-0.116754,0.034262,-0.019495,0.063877,0.064520,josh
4,-0.161875,0.143545,0.081645,0.004162,-0.002891,-0.023382,0.065750,-0.146906,0.107779,-0.044336,...,0.120897,-0.088367,0.056069,-0.051788,-0.102164,0.070031,-0.047116,0.052411,0.063778,josh


In [39]:
from sklearn.preprocessing import LabelEncoder

# Example target
y = combined['label'].tolist()
y

# # Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [40]:
y_encoded

array([1, 8, 1, 8, 8, 4, 7, 8, 5, 3, 0, 7, 3, 7, 1, 7, 8, 4, 5, 1, 8, 2,
       5, 2, 4, 1, 7, 1, 7, 1, 1, 7, 7, 5, 1, 8, 5, 6, 8, 4, 3, 2, 1, 8,
       1, 8, 6, 2, 7, 1, 1, 4, 1, 0, 7, 7, 5, 8, 0, 3, 7, 0])

In [42]:
from sklearn.model_selection import train_test_split as tts

In [110]:
X_train,X_test,y_train,y_test = tts((combined.drop('label', axis = 1)),y_encoded,test_size=0.13,random_state=6)

In [111]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((53, 128), (9, 128), (53,), (9,))

In [112]:
clf = SVC(probability=True)
clf.fit(X_train, y_train)

SVC(probability=True)

In [113]:
predictions = clf.predict(X_test)

In [114]:
predictions

array([7, 8, 1, 7, 1, 7, 8, 8, 8])

In [115]:
y_test

array([7, 8, 1, 7, 1, 2, 4, 2, 4])

In [116]:
!pip install catboost

In [117]:
from catboost import CatBoostClassifier

In [118]:
model = CatBoostClassifier(
    iterations=100,     # number of trees
    learning_rate=0.1,  # how fast to learn
    depth=6,            # depth of trees
    verbose=0           # set verbose>0 if you want training logs
)

In [119]:
model.fit(X_train,y_train)

In [123]:
model.predict(X_test).flatten().tolist(),model.predict_proba(X_test)

([7, 8, 1, 7, 1, 2, 4, 2, 4],
 array([[0.05509629, 0.05945641, 0.05057381, 0.04586188, 0.04830944,
         0.0621651 , 0.0450238 , 0.52693688, 0.10657639],
        [0.01192106, 0.01449944, 0.01371657, 0.01430718, 0.01187677,
         0.01506754, 0.01439534, 0.01519568, 0.88902043],
        [0.00981698, 0.92159819, 0.0103612 , 0.00915382, 0.00944737,
         0.0098347 , 0.01011851, 0.01041886, 0.00925036],
        [0.04506447, 0.04724492, 0.04277027, 0.04916351, 0.04204873,
         0.0898582 , 0.04152924, 0.57396414, 0.06835654],
        [0.01913863, 0.84238631, 0.01857271, 0.01827422, 0.02131465,
         0.02168713, 0.01881075, 0.01917359, 0.02064201],
        [0.10037691, 0.1141921 , 0.18032056, 0.09036449, 0.08214719,
         0.11445274, 0.0732974 , 0.11682735, 0.12802125],
        [0.06990358, 0.06450272, 0.06642204, 0.06769964, 0.37352761,
         0.11414987, 0.07093252, 0.10026401, 0.072598  ],
        [0.0832579 , 0.12029969, 0.24325649, 0.0826044 , 0.0770443 ,
         0.1

In [121]:
y_test

array([7, 8, 1, 7, 1, 2, 4, 2, 4])

In [125]:
y

['Elyse',
 'josh',
 'Elyse',
 'josh',
 'josh',
 'Joviah',
 'Richard',
 'josh',
 'Meje',
 'Jibu',
 'Don',
 'Richard',
 'Jibu',
 'Richard',
 'Elyse',
 'Richard',
 'josh',
 'Joviah',
 'Meje',
 'Elyse',
 'josh',
 'Gringo',
 'Meje',
 'Gringo',
 'Joviah',
 'Elyse',
 'Richard',
 'Elyse',
 'Richard',
 'Elyse',
 'Elyse',
 'Richard',
 'Richard',
 'Meje',
 'Elyse',
 'josh',
 'Meje',
 'Patrick',
 'josh',
 'Joviah',
 'Jibu',
 'Gringo',
 'Elyse',
 'josh',
 'Elyse',
 'josh',
 'Patrick',
 'Gringo',
 'Richard',
 'Elyse',
 'Elyse',
 'Joviah',
 'Elyse',
 'Don',
 'Richard',
 'Richard',
 'Meje',
 'josh',
 'Don',
 'Jibu',
 'Richard',
 'Don']